# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("output_data/clean_city_data.csv")
weather_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Champerico,14.30,-91.92,86.00,70,20,6.93,GT,1595869190
1,Tiksi,71.69,128.87,40.82,83,99,6.08,RU,1595868760
2,Hilo,19.73,-155.09,75.20,88,90,3.36,US,1595868757
3,Port Alfred,-33.59,26.89,57.81,68,0,14.18,ZA,1595868765
4,Tuktoyaktuk,69.45,-133.04,62.60,72,40,8.05,CA,1595868748


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# store 'Lat' and 'Lng' into locations 
locations = weather_df[["Lat", "Lng"]].astype(float)
locations.head()

,Lat,Lng
0,14.30,-91.92
1,71.69,128.87
2,19.73,-155.09
3,-33.59,26.89
4,69.45,-133.04


In [5]:
# store 'Humidity' to float
humidity = weather_df["Humidity"].astype(float)
humidity.head()

0    70.0
1    83.0
2    88.0
3    68.0
4    72.0
Name: Humidity, dtype: float64

In [6]:
# create a humidity Heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Remove rows missing data
best_cities_df = weather_df.dropna()

In [8]:
best_cities_df = best_cities_df[best_cities_df['Humidity']<=50]

In [9]:
best_cities_df = best_cities_df[best_cities_df['Max_Temp']<=75]

In [10]:
best_cities_df = best_cities_df[best_cities_df['Max_Temp']>=60]

In [11]:
best_cities_df = best_cities_df[best_cities_df['Wind_Speed']<=10]

In [12]:
best_cities_df = best_cities_df[best_cities_df['Cloudiness']<=25]

In [13]:
best_cities_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
27,Brooks,50.58,-111.89,75.00,47,0,4.00,CA,1595869192
29,Saldanha,-33.01,17.94,62.60,48,0,6.93,ZA,1595868754
52,Kamina,-8.74,24.99,70.20,29,0,4.79,CD,1595869195
101,Longyearbyen,78.22,15.64,69.80,43,0,9.17,SJ,1595868611
153,Zambezi,-13.54,23.10,66.13,31,0,5.73,ZM,1595869207
254,Caconda,-13.73,15.07,66.87,27,0,5.35,AO,1595869218
288,Calama,-22.47,-68.93,66.20,4,0,5.82,CL,1595868783
298,Karratha,-20.74,116.85,67.10,37,8,7.43,AU,1595868776
337,Östersund,63.18,14.64,68.00,49,7,4.70,SE,1595869226
511,Gravelbourg,49.88,-106.55,71.83,40,0,6.46,CA,1595869245


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = best_cities_df

In [15]:
hotel_df['Hotel_Name'] = ""
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
27,Brooks,50.58,-111.89,75.00,47,0,4.00,CA,1595869192,
29,Saldanha,-33.01,17.94,62.60,48,0,6.93,ZA,1595868754,
52,Kamina,-8.74,24.99,70.20,29,0,4.79,CD,1595869195,
101,Longyearbyen,78.22,15.64,69.80,43,0,9.17,SJ,1595868611,
153,Zambezi,-13.54,23.10,66.13,31,0,5.73,ZM,1595869207,
254,Caconda,-13.73,15.07,66.87,27,0,5.35,AO,1595869218,
288,Calama,-22.47,-68.93,66.20,4,0,5.82,CL,1595868783,
298,Karratha,-20.74,116.85,67.10,37,8,7.43,AU,1595868776,
337,Östersund,63.18,14.64,68.00,49,7,4.70,SE,1595869226,
511,Gravelbourg,49.88,-106.55,71.83,40,0,6.46,CA,1595869245,


In [16]:
# find the closest hotel to each cities coordinates

params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

# use the lat/lng to find hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)

    # convert to json
    response = response.json()
    print(json.dumps(response, indent=4, sort_keys=True))

    # since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 50.5831154,
                    "lng": -111.8980812
                },
                "viewport": {
                    "northeast": {
                        "lat": 50.5844626302915,
                        "lng": -111.8962853697085
                    },
                    "southwest": {
                        "lat": 50.5817646697085,
                        "lng": -111.8989833302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "a18e5898ba18bd0ae4200e3295f151a2e5989306",
            "name": "Heritage Inn Hotel & Convention Centre",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 102

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAMJrEHGOly9ce9GSjjorEZYDYMl5N5IPj0MKOB7YhJ5to_zp7MvSkIRTUKDDtnyf6Le-MJWbX_OdgRU1iaKQku6ySR6h4TZizF0GIyHVUdhArG7icRwNLsmh215WJxIxPGGqu2NboCI-l07PsjDuKMWdaQpSp2_2g7MG1W9htSHFPdz03jERssOkDLtLGmL7mN13bCw9aEmsAKN1BPav8dlAnw0AkK9EoS7dCkfzrxAfj9VrhAwbon4ZRt8fgNdp7w4ACIUZXaIJ7bm3M6c6mRZ4XzJraMyTGDKNwZKo-239FrwK0_Rjm8iHRifum2RZuEW4kET0fRRiXgaf1e0DHZUbgy7QroC2Ep6IAeMjqYOKmPexWgSgeBCHa4ajHkVaYRIQ2k-fMcrWuVc2gy-KurDdzxoU3ZSJ8_t3av3rNXNrETr0D78RHZQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.995413,
                    "lng": 17.97314
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.9938400197085,
                        "lng": 17.9744554802915
                    },
                    "southwest": {
                        "lat": -32.9965379802915,
 

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -8.7382328,
                    "lng": 25.0194941
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.736699919708498,
                        "lng": 25.0207810302915
                    },
                    "southwest": {
                        "lat": -8.739397880291502,
                        "lng": 25.0180830697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "b0f95b804ed57368e532ca6786954461fb9fc20e",
            "name": "Guest House la Grace",
            "photos": [
                {
                    "height": 1080,
                    "html_attributions": [
                        "<a href=\"https://maps.googl

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -13.5459509,
                    "lng": 23.1003595
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.5445922197085,
                        "lng": 23.1018170302915
                    },
                    "southwest": {
                        "lat": -13.5472901802915,
                        "lng": 23.0991190697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "cd69e2d3835f564e7025bf6cd009a3ea1f95778b",
            "name": "Royal Kutachika Lodge",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1836,
                 

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAKGwYWQ9a33x8QW5pqi-lSwAc3UV59VUsywpKHiHfb0hFklVdXCK8HMO7JtfNO__PuYAAZ2fJkQgoyjmOW6riKA9-qcuS2Eo3iCnBPElTojtAos8lFOoxJWsh_XbADkwZev7GO3y70SpDkkviBFFUBJYgVzwVlq9rJdUNr4yCtuCYBfz5HI_FFeVGrbmzj5UCHO0U30sstm1QJWzuBzR9jdYyglPpFuVw0XLccFvglj3giOYF648RyynrcIXVVKYP4n1uAhBkzHxE7DfqKOtbhjBJh2Iudfu3cFiOGXz9UeH_IsEs-C6HWFIDjzaVrPaqSbsXBWYtuMb1F0LxWZu798Oq1FYeBr11wQDWZu3cir_fUXahbHnJIXxJADwoQilphIQIKGQk00dxHouMFUTXsJcmhoUkRAvw87eM2B-piwYIEmgUmTbMUo",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.4892254,
                    "lng": -68.9226497
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.4879244197085,
                        "lng": -68.920

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAIZcKwbyIU6x-BYpt9pNPS4Z3Wh1Hy5QjFntBUsP65HlnjH_R5OXv3o2V4pfzjnDrjwVIxwMOZtiA4m4568VYj9-s8O5yE5jiHlaTStadLUiP6p_J0McHdT2S39RqTkcQm9iPeJLRUGIV2Hk8d4DeO0CszJM8l8mJz18RU3AET7zQ1j8GP6XKPfBDayDjIe1EoZ588omtxlGhtGLGa6D163g7fgvuORcbF-o5aBxCm5cNb_N4qWRtZhC4l3cFTbjRywdwFT2MTJblCOoMvfq-J24SpuvvHJRdNNJ7xGrUJv-iKqPCrZ-xk2ofxpYcd27B2MAlxZ8DhdW2E2SdoM6Yo3f_952Cs193iIKrs86TAi_EQ24WbW9A49jKdziLhLwIRIQD8W_sxdTOlxJhQOx6x8y2hoUAX7X7wttUnTWKytCo8LS_eeiIhw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.7322255,
                    "lng": 116.8467438
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.7308124197085,
                        "lng": 116.8481896302915
                    },
                    "southwest": {
                        "lat": -20.73351038029

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAH1Avm4E8XcPD8XXo2WczrIAxI2PEjClXowpqSANxUXn2l5I4CwqK4vlQcJjGZS-gNDDgsySyRLdfqOdoWeQb81VdWAyyB9j1oxth8T-nL-s2O57mzwOcEQHYdgGFtvj93yMW0KzFQn7jrtQj1aXhwBR21Mt_0qg2PwSk25Ug2JwCBQJqfU2aDLdy7YeATh5Pu-G7x7POdj27hu9GjdsiRHSfahceiS2b8KC68wNAJRDlkBSuyZRuwucr6rb_fgj4VrlBs5i07V0kyyB6FJds5MTnhEOUprFA_pt-eBtIMgRHfEInse96bYNK_0_hVZ7XHBe6fo4g29DdH_jZmjIfY_J6wZGAYk3MeVrQ3YbOFw7SSpY6jbKCbdT5dbqDWx_oBIQG4-5GylHA19QQ9u-4XOaohoU3a_07xLhnw413fZxe0AdUO39qMI",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 63.1751878,
                    "lng": 14.6405984
                },
                "viewport": {
                    "northeast": {
                        "lat": 63.17660818029151,
                        "lng": 14.6419336802915
                    },
                    "southwest": {
                        "lat": 63.1739102197085,


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 49.87261849999999,
                    "lng": -106.5583107
                },
                "viewport": {
                    "northeast": {
                        "lat": 49.8741022302915,
                        "lng": -106.5569656697085
                    },
                    "southwest": {
                        "lat": 49.8714042697085,
                        "lng": -106.5596636302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "8e253dae99c776907e92da4b4bdf9453e3a4dd10",
            "name": "Historic Bishops Residence Bed & Breakfast",
            "photos": [
                {
                    "height": 4032,
                    "html_attributions": [
                      

In [17]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
27,Brooks,50.58,-111.89,75.00,47,0,4.00,CA,1595869192,Heritage Inn Hotel & Convention Centre
29,Saldanha,-33.01,17.94,62.60,48,0,6.93,ZA,1595868754,Blue Bay Lodge
52,Kamina,-8.74,24.99,70.20,29,0,4.79,CD,1595869195,Guest House la Grace
101,Longyearbyen,78.22,15.64,69.80,43,0,9.17,SJ,1595868611,"Radisson Blu Polar Hotel, Spitsbergen"
153,Zambezi,-13.54,23.10,66.13,31,0,5.73,ZM,1595869207,Royal Kutachika Lodge
254,Caconda,-13.73,15.07,66.87,27,0,5.35,AO,1595869218,
288,Calama,-22.47,-68.93,66.20,4,0,5.82,CL,1595868783,Park Hotel Calama
298,Karratha,-20.74,116.85,67.10,37,8,7.43,AU,1595868776,ibis Styles Karratha
337,Östersund,63.18,14.64,68.00,49,7,4.70,SE,1595869226,Best Western Hotel Gamla Teatern
511,Gravelbourg,49.88,-106.55,71.83,40,0,6.46,CA,1595869245,Historic Bishops Residence Bed & Breakfast


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))